In [31]:
#pip install git+https://github.com/openai/whisper.git
#pip install ffmpeg
import whisper

# **Pre-processing**

# **Speech to Text**

In [32]:
test_record = "records/audio1.wav"

In [33]:
model = whisper.load_model("small")
result = model.transcribe(test_record, fp16=False)
print(result["text"])

 Siempre me ha preocupado enormemente ser entendido y aceptado. Pero he aprendido que agradar o contentar a todo el mundo es imposible. Por más que hagamos, nos esforcemos o justifiquemos, siempre habrá alguien que nos busque y a quien no le guste lo que hacemos. En el poder de confiar en ti, comparto de la forma más clara y directa todo lo que he vivido y aprendido. No quiero guardarme nada para mí, porque sé que el cambio es posible. También puedes hacerlo tú, independientemente del lugar en el que te encuentres. Recuerda, las puertas del cambio interno están siempre abiertas para todo el que se decida pasar por ellas.


# **Text to Speech**

In [35]:
import pyttsx3

engine = pyttsx3.init()
engine.say(result["text"])
engine.runAndWait()
engine.stop()